# Convolutional neural nets lab

# 1 Let's run the santa example in Keras

In [2]:
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(1)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# directory path
train_data_dir = 'data/train'
test_data_dir = 'data/validation'

# get all the data in the directory data/validation (132 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, 
        target_size=(64, 64), batch_size=132) 

# get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, 
        target_size=(64, 64), batch_size=790)

# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

In [ ]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images_orig shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

In [ ]:
# Reshape the training and test examples 
train_img = train_images.reshape(train_images.shape[0], -1)  # The "-1" makes reshape flatten the remaining dimensions
test_img = test_images.reshape(test_images.shape[0], -1)

# Standardize data to have feature values between 0 and 1.
train_x = train_img/255.
test_x = test_img/255.

print ("train_x's shape: " + str(train_img.shape))
print ("test_x's shape: " + str(test_img.shape))

In [ ]:
# Reshape the labels
train_y = train_labels[:,0]
train_y = np.reshape(train_y, (790,1))

test_y = test_labels[:,0]
test_y = np.reshape(test_y, (132, 1))

print ("train_y's shape: " + str(train_y.shape))
print ("test_y's shape: " + str(test_y.shape))

In [ ]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) #2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_x,
                    train_y,
                    epochs=50,
                    batch_size=34,
                    validation_data=(test_x, test_y))

# 2. Let's see how we can perform data augmentation in Keras

In [ ]:
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
from keras.preprocessing import image
import os

In [ ]:
santa_dir= 'data/train/santa'
fnames = [os.path.join(santa_dir, fname) for
     fname in os.listdir(santa_dir)]
img_path = fnames[3]
img = image.load_img(img_path, target_size=(64, 64))

arrayimg = image.img_to_array(img) 
arrayimg = arrayimg.reshape((1,) + arrayimg.shape) 
i=0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 8 == 0:
        break
plt.show()

# 2. Conv net without data augmentation

In [51]:
# directory path
train_data_dir = 'data/train'
test_data_dir = 'data/validation'

# get all the data in the directory data/validation (132 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, 
        target_size=(64, 64), batch_size=132) 

# get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, 
        target_size=(64, 64), batch_size=790)

# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

Found 132 images belonging to 2 classes.
Found 790 images belonging to 2 classes.


In [57]:
# Reshape the training and test examples 
train_img = train_images.reshape(train_images.shape[0], -1)  # The "-1" makes reshape flatten the remaining dimensions
test_img = test_images.reshape(test_images.shape[0], -1)

# Standardize data to have feature values between 0 and 1.
train_x = train_images/255.
test_x = test_images/255.

print ("train_x's shape: " + str(train_img.shape))
print ("test_x's shape: " + str(test_img.shape))

np.shape(train_x)

train_x's shape: (790, 12288)
test_x's shape: (132, 12288)


(790, 64, 64, 3)

In [58]:
# Reshape the labels
train_y = train_labels[:,0]
train_y = np.reshape(train_y, (790,1))

test_y = test_labels[:,0]
test_y = np.reshape(test_y, (132, 1))

print ("train_y's shape: " + str(train_y.shape))
print ("test_y's shape: " + str(test_y.shape))

train_y's shape: (790, 1)
test_y's shape: (132, 1)


In [59]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Conv2D(32, (4, 4), input_shape=(64 ,64,  3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (4, 4)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (4, 4)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [61]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [63]:
history = model.fit(train_x,
                    train_y,
                    epochs=20,
                    batch_size=24,
                    validation_data=(test_x, test_y))

Train on 790 samples, validate on 132 samples
Epoch 1/20
790/790 [==============================] - 7s 9ms/step - loss: 0.1017 - acc: 0.9772 - val_loss: 0.4710 - val_acc: 0.8636
Epoch 2/20
790/790 [==============================] - 7s 9ms/step - loss: 0.0286 - acc: 0.9937 - val_loss: 1.0134 - val_acc: 0.7727
Epoch 3/20
790/790 [==============================] - 7s 9ms/step - loss: 0.0808 - acc: 0.9747 - val_loss: 0.4252 - val_acc: 0.8939
Epoch 4/20
790/790 [==============================] - 8s 10ms/step - loss: 0.0325 - acc: 0.9911 - val_loss: 0.4709 - val_acc: 0.8712
Epoch 5/20
790/790 [==============================] - 7s 9ms/step - loss: 0.0799 - acc: 0.9835 - val_loss: 0.5040 - val_acc: 0.8712
Epoch 6/20
790/790 [==============================] - 8s 10ms/step - loss: 0.0184 - acc: 0.9911 - val_loss: 0.5853 - val_acc: 0.8636
Epoch 7/20
790/790 [==============================] - 8s 10ms/step - loss: 0.0773 - acc: 0.9810 - val_loss: 0.4794 - val_acc: 0.8636
Epoch 8/20
790/790 [=======

# 3. Convnet with data augmentation now!

In [64]:
# directory path
train_data_dir = 'data/train'
test_data_dir = 'data/validation'

# get all the data in the directory data/validation (132 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, 
        target_size=(64, 64), batch_size=132) 

# get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, 
        target_size=(64, 64), batch_size=790)

# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

Found 132 images belonging to 2 classes.
Found 790 images belonging to 2 classes.


In [65]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

In [66]:
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(64, 64),
    batch_size=24,
    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
    'data/validation',
    target_size=(64, 64),
    batch_size=24,
    class_mode='binary')

Found 790 images belonging to 2 classes.
Found 132 images belonging to 2 classes.


In [67]:
np.shape(train_images)

(790, 64, 64, 3)

In [68]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Conv2D(32, (4, 4), input_shape=(64 ,64,  3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (4, 4)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (4, 4)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [69]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [70]:
history_2 = model.fit_generator(
      train_generator,
      steps_per_epoch=10,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=10)

Epoch 1/20
10/10 [==============================] - 15s 2s/step - loss: 0.6962 - acc: 0.5000 - val_loss: 0.6613 - val_acc: 0.4474
Epoch 2/20
10/10 [==============================] - 9s 869ms/step - loss: 0.6489 - acc: 0.6369 - val_loss: 0.6103 - val_acc: 0.7237
Epoch 3/20
10/10 [==============================] - 11s 1s/step - loss: 0.5658 - acc: 0.7250 - val_loss: 0.5743 - val_acc: 0.7807
Epoch 4/20
10/10 [==============================] - 11s 1s/step - loss: 0.4698 - acc: 0.8625 - val_loss: 0.5118 - val_acc: 0.8114
Epoch 5/20
10/10 [==============================] - 9s 907ms/step - loss: 0.3976 - acc: 0.8412 - val_loss: 0.7827 - val_acc: 0.7588
Epoch 6/20
10/10 [==============================] - 10s 1s/step - loss: 0.4441 - acc: 0.8167 - val_loss: 0.4578 - val_acc: 0.8114
Epoch 7/20
10/10 [==============================] - 10s 984ms/step - loss: 0.3684 - acc: 0.8667 - val_loss: 0.5604 - val_acc: 0.7851
Epoch 8/20
10/10 [==============================] - 8s 760ms/step - loss: 0.4245 - 


## 8. Conclusion

A deeper network drastically improves the performance on our data set with Santa images!
